# Stack Exchange Question Classifier

Stack Exchange é um repositório de informações que possui 105 tópicos diferentes e cada tópico tem uma acervo de perguntas que foram feitas e respondidas por membros experientes da comunidade do StackExchange. Os tópicos são tão diversos quanto viagens, culinária, programação, engenharia e fotografia. Foram escolhidos 10 categoria de tópicos (gis, security, photo, mathematica, unix, wordpress, scifi, electronics, android, apple) e dada uma pergunta e um trecho, sua tarefa é identificar quais dentre os 10 tópicos a que pertence.

In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

### 1. Leitura e formatação dos dados

In [2]:
data = pd.read_json('training.json', orient='columns')

Para executar algoritmos de aprendizado de máquina, precisamos converter os arquivos de texto em vetores de features numéricas. Nós estaremos usando o modelo 'bag of words' para o nosso exemplo. Resumidamente, nós segmentamos cada arquivo de texto em palavras (dividido pelo espaço) e contamos o número de vezes que cada palavra ocorre em cada documento e finalmente atribuímos a cada palavra um ID inteiro. Cada palavra única no nosso dicionário irá corresponder a uma característica 

In [3]:
df = CountVectorizer(stop_words='english', strip_accents='unicode')

data_test = data['excerpt']
y = data['topic']

X_train = df.fit_transform(data_test)
X_train.shape


(20219, 29135)

Selecionando cada tipo de variável (features e target) do conjunto de dados. Apenas contar o número de palavras em cada documento tem um problema: ele dará mais peso a documentos mais longos do que documentos mais curtos. Para evitar isso, podemos usar a frequência (TF - Term Frequencies) e, também, podemos até reduzir o peso de palavras mais comuns (TF-IDF - Term Frequency times inverse document frequency).

In [4]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train)
X_train_tfidf.shape

(20219, 29135)

Existem vários algoritmos que podem ser usados para classificação de texto, vamos usar Naive Bayes do sklearn.

### 2. Treinamento

In [5]:
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_tfidf, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

### 3. Teste

In [6]:
data_test_json = pd.read_json('input/input00.txt', orient='columns')

In [7]:
X = data_test_json['excerpt']

X_test = df.transform(X)
X_test_tfidf = tfidf_transformer.transform(X_test)


In [8]:
print(X_train_tfidf.shape)
print(X_test_tfidf.shape)
y_pred = naive_bayes.predict(X_test)

(20219, 29135)
(15032, 29135)


In [44]:
y_true = pd.read_csv('output/output00.txt', header=None, names=['labels'])

print(np.mean(y_pred == np.array(y_true['labels']) ))

0.8495210218201171
